In [20]:
import xlrd
import re
import json
import jieba
import pandas as pd
import pickle
from collections import Counter
from zhon.hanzi import punctuation
jieba.enable_paddle()

Paddle enabled successfully......


In [ ]:
people_congress = []
data1 = xlrd.open_workbook("government_report1.xlsx")
table1 = data1.sheet_by_index(1)
nrows = table1.nrows
ncols = table1.ncols
for row in range(1, nrows):
    excel_rows = []
    for col in range(ncols):
        cell_value = table1.cell(row,col).value
        excel_rows.append(cell_value)
    people_congress.append(excel_rows)
people_congress = pd.DataFrame(people_congress)
len(people_congress)
#会议文本位于people_congress[i][6]

In [ ]:
party_congress = []
data2 = xlrd.open_workbook("party_congress.xlsx")
table2 = data2.sheet_by_index(0)
nrows = table2.nrows
ncols = table2.ncols
for row in range(1, nrows):
    excel_rows = []
    for col in range(ncols):
        cell_value = table2.cell(row,col).value
        excel_rows.append(cell_value)
    party_congress.append(excel_rows)
len(party_congress)

people_congress = pd.DataFrame(people_congress)
people_congress.to_pickle("./people_congress.pkl")#存储没有清洗的元数据
#会议文本位于party_congress[i][7]

In [ ]:
punctuation_str = punctuation #导入标点符号库
stopwords = [line.strip() for line in open('stopwords.txt', 'r', encoding = 'utf-8').readlines()]
#导入停顿词库
jieba.load_userdict("dictionary.txt") #导入自定义词典（未使用，可按照词典筛选词）

In [ ]:
#党代清洗
party_text = [] #去除符号、标点等候保留的纯文本
for year in range(len(party_congress)):
    a = party_congress[year][7]
    a = a.replace('\n','').replace('\r','').replace(' ','') #去除换行，空格
    for i in punctuation:
        a = a.replace(i,'') #去除标点符号
    party_text.append(a)

In [ ]:
#创建观感良好的data frame
year = []
for i in range(len(party_congress)):
    t = str(party_congress[i][0])
    r = t.rstrip('0').strip('.')
    year.append(r)

number = []
for i in range(len(party_congress)):
    t = str(party_congress[i][1])
    r = t.rstrip('0').strip('.')
    number.append(r) 

sub_number = []
for i in range(len(party_congress)):
    t = str(party_congress[i][3])
    r = t.rstrip('0').strip('.')
    sub_number.append(r)  

doctype = []
for i in range(len(party_congress)):
    t = str(party_congress[i][5])
    r = t.rstrip('0').strip('.')
    doctype.append(r)  

party_list = list(zip(year, number, sub_number, doctype, party_text))

import pandas as pd
pd.set_option('max_colwidth',150)
data_df=pd.DataFrame(party_list, columns=['year','number','sub_number','doctypr','report'])

In [3]:
#将不同年限的报告整合在一起
def ab(data_df):
    return','.join(data_df.values)
data_df2 = data_df.groupby('year')['report'].apply(ab)
data_df2 = data_df2.reset_index()
data_df2 = data_df2.drop(0)
data_df2 = data_df2.reset_index() 

data_df2.to_pickle("./data_df2.pkl")#存储清洗后的数据
#删掉第一行后出了错配问题，resetindex后解决，虽然出现了多余一列，但在后续匹配data_final时没有出现问题

In [8]:
#分词函数
def jiebait(text):
    seglist = jieba.cut(text, cut_all = False)
    year_word = []
    for word in seglist:
        if word !='\t':
            if word not in stopwords:
                year_word = ['  '.join(word)]
    filter_seglist = [fil for fil in year_word if len(fil) >= 2]
    return filter_seglist 
#似乎不太好用，尝试用CountVectorize的方法

In [10]:
#创建适合CountVectorize的目标数据格式
report = []
for i in range(len(data_df2)):
    all_list = ['  '.join(jieba.cut(data_df2['report'][i],cut_all = False))]
    report.append(all_list)
data_final = dict(zip(data_df['year'],report))
data_final_df = pd.DataFrame.from_dict(data_final).transpose()
data_final_df.columns = ['report']

data_final_df.to_pickle("./data_final.pkl") #存储分词后的数据

NameError: name 'data_df2' is not defined

In [13]:
#将分词后数据转化为向量
from sklearn.feature_extraction.text import CountVectorizer
count_vec = CountVectorizer(stop_words=stopwords,min_df=2)
x_count = count_vec.fit_transform(data_final_df.report)
data_dtm = pd.DataFrame(x_count.toarray(),columns=count_vec.get_feature_names())
data_dtm.index = data_final_df.index
data_dtm

pickle.dump(count_vec, open("./cv_stop.pkl", "wb")) #存储cv数据本身作为语料库
data_dtm.to_pickle("./dtm.pkl") #documenttermmatrix数据

NameError: name 'stopwords' is not defined

In [ ]:
#生成提次数最多得100个词及其词频
import pickle
data = pd.read_pickle('./dtm.pkl')
data = data.transpose()
data.head()

top_dict = {}
for c in data.columns:
    top = data[c].sort_values(ascending=False).head(100)
    top_dict[c]= list(zip(top.index, top.values))

top_dict #这个记得保存一下

In [ ]:
#并将前30个词打印
for year, top_words in top_dict.items():
    print(year)
    print(', '.join([word for word, count in top_words[0:29]]))
    print('---')

In [ ]:
#tutorial这部分教程是希望将top30words加入stopword，但是comedian数据集和政府工作报告数据集存在区别，我保留了这段code，但没有执行，有必要可以执行
from collections import Counter
# Let's first pull out the top 30 words for each comedian
words = []
for year in data.columns:
    top = [word for (word, count) in top_dict[year]]
    for t in top:
        words.append(t) #将频率最高得词进行保存得方法，但注意结果不是按年份，修改代码如下:

words = []
for year in data.columns:
    top = [word for (word, count) in top_dict[year]]
    year = []
    for t in top:
        year.append(t)
    word.append(year)

In [ ]:
#wordcloud
!pip install WordCloud

In [ ]:
from wordcloud import WordCloud
wc = WordCloud(stopwords=stopwords, background_color="white", colormap="Dark2",
               max_font_size=150, random_state=42)

In [ ]:
import matplotlib.pyplot as plt
# Allows charts to appear in the notebook
%matplotlib inline

In [ ]:
data_clean = pd.read_pickle('./data_df2.pkl')

In [ ]:
plt.rcParams['figure.figsize'] = [16, 6]

a = list(range(1977,2021))
year_num = []
for i in range(len(a)):
    year.append(str(a[i]))
year_num

In [ ]:
# Create subplots for each year
for index, year in enumerate(data.columns):
    wc.generate(data_clean.report[index])
    plt.subplot(11, 4, index+1)
    plt.imshow(wc, interpolation="bilinear")
    plt.axis("off")
    plt.title(year_num[index])
    
plt.show() #这里的wordcloud画不出来，先看下topic model

In [21]:
data = pd.read_pickle('./dtm.pkl')

In [18]:
!pip install gensim

In [22]:
from gensim import matutils, models
import scipy.sparse

In [23]:
tdm = data.transpose()
sparse_counts = scipy.sparse.csr_matrix(tdm)
corpus = matutils.Sparse2Corpus(sparse_counts)
cv = pickle.load(open("./cv_stop.pkl", "rb"))#导入基本语料库

In [27]:
id2word = dict((v, k) for k, v in cv.vocabulary_.items())

In [ ]:
lda = models.LdaModel(corpus=corpus, id2word=id2word, num_topics=20, passes=100)
lda.print_topics() #这里给出了针对所有文档资料的topic

In [24]:
from gensim import corpora, models, similarities
from gensim.models import LdaModel
from gensim.models import ldaseqmodel
from gensim.corpora import Dictionary, bleicorpus
import numpy
from gensim.matutils import hellinger

In [25]:
#根据时间变化的topic
sparse_counts2 = scipy.sparse.csr_matrix(data)
corpus2 = matutils.Sparse2Corpus(sparse_counts2)

In [ ]:
c = [1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,1,]
time_slice = c
ldaseq = ldaseqmodel.LdaSeqModel(corpus=corpus, id2word=id2word, time_slice=time_slice, num_topics=3)

C:\Users\Abel\Anaconda3\lib\site-packages\gensim\models\ldaseqmodel.py:293: RuntimeWarning: divide by zero encountered in double_scalars
  convergence = np.fabs((bound - old_bound) / old_bound)
